<a href="https://colab.research.google.com/github/mabittar/Learn/blob/master/Covid_Counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#to use Covid Counter from https://github.com/sudharshan-ashok/pycovid
!pip install pycovid


In [0]:
#import modules
import pandas as pd
from datetime import datetime, date
import re
import numpy as np
import pandas as pd

In [0]:
#import database for Covid confirmed cases
from pycovid import pycovid
confirmed_cases_df = pycovid.getCovidCases(casetype=['confirmed'], cumsum=False)

In [0]:
#import country population
population_url = "https://raw.githubusercontent.com/julianbruegger/corona-display/master/population.csv"
colnames_remap= {"Country":"country",
                 "Country Code":"alpha-3",
                 "2018":"population" 
                }
col_drop = ["country"]
population_df = pd.read_csv(population_url, encoding= 'unicode_escape', sep=";").rename(columns=colnames_remap).drop(columns=col_drop)
#population_df.head(10)
#print(population_df.columns)

In [0]:
#rename and reorganize columns
renamed_columns_map = {
    "country_region": "country",
    "province_state": "location",
    "lat": "latitude",
    "long": "longitude",
    "type": "status"
    }

cols_to_drop = ["location", "latitude", "longitude", "name", "status"]

df = (confirmed_cases_df.rename(columns=renamed_columns_map).drop(columns=cols_to_drop))
#df.head(10)

In [9]:
#define important dates
last_date = df['date'].max().date()#.strftime("%Y%m%d")
#last_date = int(last_date)
print(last_date)

today = date.today()
print(today)

2020-03-22
2020-03-23


In [0]:
#Group country with most confirmed cases
most_df = df.groupby(['alpha-3','country']).sum().reset_index()
most_df = most_df.sort_values(by='cases', ascending=False).head(10)
most_df = most_df.merge(population_df, on='alpha-3', how='left')
#most_df.head(10)

In [0]:
# Create a function that takes inputs, cases and population to calculare % infected
def pop_cases(cases, population):
    # returns the % between cases and population
    return cases / population

# Create a variable that is the output of the function
most_df['percentage'] = pop_cases(most_df['cases'], most_df['population'])

#most_df.head(10)

In [0]:
dff = most_df.sort_values(by='percentage', ascending=False).head(10)
#dff

In [13]:
#filter most increase cases in last date of data
increase_df = df[df['date'].eq(last_date)].sort_values(by='cases', ascending=False).head(10)
#increase_df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/__init__.py:1350: FutureWarning:

Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.



In [14]:
#Display 1o country in number of cases
country1 = most_df['country'].iloc[0]
#Display increased cases
increase_country = increase_df['country'].iloc[0]
new_cases = increase_df['cases'].iloc[0]
cases1 = most_df['cases'].iloc[0]
percent_pop = most_df['percentage'].iloc[0]
print("The country with most confirmed cases in {:%d/%m/%Y} is {}. With {:,} cases confirmed. ".format(last_date,country1,cases1))
print("It´s represent {:.3%} of {} population".format(percent_pop,country1))
print("The country with most increase in {:%d/%m/%Y} is {} with {} new cases.".format(last_date,increase_country,new_cases))


The country with most confirmed cases in 22/03/2020 is Italy. With 59,138 cases confirmed. 
It´s represent 0.098% of Italy population
The country with most increase in 22/03/2020 is Italy with 5560 new cases.


In [0]:
"""""
import tkinter as tk

def popupmsg(msg):
    popup = tk.Toplevel()
    popup.title("!")
    label = tk.Label(popup, text=msg) #Can add a font arg here
    label.pack(side="top", fill="x", pady=10)
    B1 = tk.Button(popup, text="Okay", command = popup.destroy)
    B1.pack()
    popup.mainloop()

popupmsg('The country with most confirmed cases in {:%d/%m/%Y} is {}. With {:,} cases confirmed./p'
        'It´s represent {:.3%} of {} population  /p'
        'The country with most increase in {:%d/%m/%Y} is {} with {} new cases.'.format(last_date,country1,cases1,percent_pop,country1,last_date,increase_country,new_cases)
        )
"""""